In [1]:
import os
import time
import numpy
import tensorflow as tf
import input_data
import vw_c3d_newnetwork
import vw_c3d_tools
import math
import numpy as np
import input_data_for_extract_feature as input_data
from six.moves import xrange

BATCH_SIZE = 16
gpu_num = 1
#MAX_STEPS = 10000
NUM_FRAMES_PER_CLIP = 16
N_CLASSE = 9
CROP_SIZE = 112
CHANNELS = 3
MAX_EPOCHS = 100

model_filename = './feature_extra_models/model-100'
test_list_file = 'list/test.list'
unit_of_clip=2
data_dir = "./fc6_bin_data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)  


def placeholder_inputs(batch_size):
    images_placeholder = tf.placeholder(tf.float32, shape=(BATCH_SIZE,
                                                           NUM_FRAMES_PER_CLIP,
                                                           CROP_SIZE,
                                                           CROP_SIZE,
                                                           CHANNELS))
    labels_placeholder = tf.placeholder(tf.int64, shape=[BATCH_SIZE, N_CLASSE])
    return images_placeholder, labels_placeholder

def train():
    num_test_videos = len(list(open(test_list_file,'r')))
    print("Number of test videos={}".format(num_test_videos))
    with tf.Graph().as_default():
        #构建数据和标签的place_holder 
        images_placeholder, labels_placeholder = placeholder_inputs(BATCH_SIZE * gpu_num)
        
        #构建logits、loss、accuracy、train_op
        fc6 = vw_c3d_newnetwork.C3D_MODEL_fc6(images_placeholder,N_CLASSE)

        #存储模型
        saver = tf.train.Saver(tf.global_variables())

        #初始化所有变量，并开启会话
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        saver.restore(sess, model_filename)
        
        next_batch_start = 0
        all_steps = int((num_test_videos - 1) / (BATCH_SIZE * gpu_num) + 1)
        file_index = 0
        fc6_batch_size=BATCH_SIZE
        
        for step in xrange(all_steps):
            np_arr_data, np_arr_label, next_batch_start, _, _ = input_data.read_clip_and_label(
                        test_list_file,fc6_batch_size,unit_of_clip,start_pos=next_batch_start)     
            fc6_of_batch=[]
            print("np_arr_data",np.shape(np_arr_data))
            batch_seq_arr = np_arr_data.reshape((fc6_batch_size * unit_of_clip, 16,112,112,3))
            print("batch_seq_arr",np.shape(batch_seq_arr))

            for batch_unit_index in range(0,unit_of_clip):
                temp_batch_data = batch_seq_arr[batch_unit_index*fc6_batch_size:batch_unit_index*fc6_batch_size + fc6_batch_size]
                batch_seq_fc6 = sess.run(fc6,feed_dict={images_placeholder:temp_batch_data})
                fc6_of_batch.append(batch_seq_fc6)
#                 print("np.shape(batch_seq_fc6)",np.shape(batch_seq_fc6))
#                 print("np.shape(fc6_of_batch)",np.shape(fc6_of_batch))
            fc6_of_batch = np.array(fc6_of_batch)
            fc6_feature_batch = fc6_of_batch.reshape((fc6_batch_size,unit_of_clip * 4096))
#             print("np.shape(fc6_feature_batch)",np.shape(fc6_feature_batch))
#             print("np_arr_label",np_arr_label)
#             print(fc6_feature_batch)

            for batch_index in range(fc6_batch_size):            
                try:
                    data = fc6_feature_batch[batch_index]
                    label = np_arr_label[batch_index]
                    file_index += 1
                    filename = "%s/%08d_%02d.bin" % (data_dir, file_index, label)
                    print("data-->",data)
                    print("label-->",label)
                    print("filename",filename)
                    with open(filename, 'wb') as f:
                        f.write(data)    
                except IOError as e:
                    print('Skip it!\n')     

    print("Done!")

In [2]:
train()

Number of test videos=538
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/141...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/142...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/143...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/144...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/145...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/146...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/147...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/148...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/149...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/150...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/151...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/152...
Loading a video clip from /samba/HDD/dataset_elec_

data--> [-0.32520792  0.          1.33844566 ..., -0.87905484  0.          0.        ]
label--> 0
filename ./bin_data/00000033_00.bin
data--> [-0.74430853  0.         -0.10520828 ...,  1.09783268  0.          0.        ]
label--> 0
filename ./bin_data/00000034_00.bin
data--> [-0.74430853  0.         -0.69270164 ..., -0.87905484  0.          0.        ]
label--> 0
filename ./bin_data/00000035_00.bin
data--> [-0.74430853  0.          0.12790793 ...,  1.40605116  0.          0.        ]
label--> 0
filename ./bin_data/00000036_00.bin
data--> [-0.74430853  0.         -0.69270164 ..., -0.87905484  0.          0.        ]
label--> 0
filename ./bin_data/00000037_00.bin
data--> [ 0.42204028  0.          0.10688859 ..., -0.87905484  0.          0.        ]
label--> 0
filename ./bin_data/00000038_00.bin
data--> [-0.14873475  0.         -0.69270164 ..., -0.87905484  0.          0.        ]
label--> 0
filename ./bin_data/00000039_00.bin
data--> [ 1.96030188  0.         -0.69270164 ..., -0.87905484 

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/164...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/165...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/166...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/167...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/168...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/169...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/170...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/171...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/172...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/173...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/174...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/175...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/176...

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/152...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/153...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/154...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/155...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/156...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/157...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/158...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/159...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/160...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/161...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/162...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/163...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/164...

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/200...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/141...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/142...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/143...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/144...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/145...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/146...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/147...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/148...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/149...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/150...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/151...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/152...

...

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/152...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/153...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/154...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/155...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/156...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/157...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/158...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/159...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/160...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/161...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/162...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/163...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/164...

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_07/200...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/141...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/142...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/143...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/144...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/145...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/146...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/147...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/148...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/149...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/150...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/151...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/152...

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/188...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/189...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/190...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/191...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/192...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/193...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/194...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/195...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/196...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/197...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/198...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/199...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/200...

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/175...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/176...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/177...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/178...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/179...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/180...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/181...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/182...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/183...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/184...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/185...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/186...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/187...

ValueError: cannot reshape array of size 12042240 into shape (32,16,112,112,3)